## Plotly testing

In [63]:
import pandas as pd
import numpy as np
import scipy as sp
import plotly.figure_factory as ff
import plotly.graph_objs as go
import plotly.plotly as py
import plotly.tools as pytools
import matplotlib.pyplot as plt
import datetime
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
pytools.set_credentials_file(username='hrishioa', api_key='xFpLRfOD7le9KEQU2DuE')

In [2]:
# Uncomment to view as spreadsheet
# df_temp = pd.read_csv("calibrated_with_dates.csv",nrows=10)
# sample_data_table = ff.create_table(df.head())
# py.iplot(sample_data_table, filename='sample-data-table')

In [89]:
def plotDataFrame(datafile, filename):
    
    df = pd.read_csv(datafile)
    
    if 'DateTime' not in df:
        df['DateTime'] = [datetime.datetime.fromtimestamp(v) for v in df['Time']]

    def slice(charge, time, start = 0 , sd = 60):
        start_pos = start
        end_pos = start_pos+1

        while end_pos < len(time) and charge[end_pos-1] == charge[end_pos]: 
            start_pos = end_pos
            end_pos = start_pos + 1

        while True:
            if end_pos >= len(time) or end_pos >= len(charge):
                break
            if abs((time[end_pos] - time[end_pos-1])-60) >= sd or charge[end_pos-1] < charge[end_pos]:
                if end_pos > start_pos+1:
                    break
                start_pos = end_pos
            end_pos += 1            
        return start_pos, end_pos

    def get_slices(charge, time):
        slices = []
        s = 0
        while s < len(time):
            start, end = slice(charge,time,s)
            if(end >= len(time)):
                slices.append((start,len(time)-1))
            else:
                slices.append((start,end))
            s = end
        return slices

    slices = get_slices(df['Charge'], df['Time'])
    buttons = list()

    for i in xrange(1,len(df.keys())-1):
        if i==3: continue
        opacity = [1 if j==2 or j==i-1 else 0.1 for j in xrange(0,len(df.keys())-1)]
        label = df.keys()[i]
        buttons.append(dict(label=label,method="update",args=[{'opacity': opacity}]))
    
    for i in xrange(0, len(slices)):
        buttons.append(dict(
            label=("Window %d" % i),
            method="relayout",
            args = [{'xaxis.range':[df['DateTime'][slices[i][0]],df['DateTime'][slices[i][1]]]}]
        ))
        
    

    updatemenus = list([
        dict(buttons=buttons
        )
    ])
    
    traces = []
    
    for i in xrange(1, len(df.keys())-1):
        traces.append(go.Scatter(
            x=df['DateTime'],y=df[df.keys()[i]],mode='lines',name=df.keys()[i], yaxis="y%d" % i))

    layout = go.Layout(title='Complete Equalised Plot',
                       margin=dict(l=200),
                       updatemenus=updatemenus,
                       xaxis=dict(
                            rangeslider=dict(),
                            type='DateTime'
                       ),
                       yaxis=dict(autorange=True,showgrid=False,showticklabels=False,),
                       yaxis2=dict(autorange=True,showgrid=False,showticklabels=False,overlaying='y'),
                       yaxis3=dict(autorange=True,showticklabels=False,overlaying='y'),
                       yaxis4=dict(autorange=True,showgrid=False,showticklabels=False,overlaying='y'),
                       yaxis5=dict(autorange=True,showgrid=False,showticklabels=False,overlaying='y'),                  
                       yaxis6=dict(autorange=True,showgrid=False,showticklabels=False,overlaying='y'),                  
                       yaxis7=dict(autorange=True,showgrid=False,showticklabels=False,overlaying='y'),                  
                       yaxis8=dict(autorange=True,showgrid=False,showticklabels=False,overlaying='y'),                  
                      )
    fig = go.Figure(data=traces, layout=layout)
    return slices,py.iplot(fig, filename=filename)

In [95]:
slices, ploturl = plotDataFrame("uncalibrated_with_dates.csv", filename="battery_plot2")
ploturl

In [91]:
slices, ploturl = plotDataFrame("calibrated_with_dates.csv", filename="battery_plot2")

In [80]:
len(df.keys())

10

In [92]:
ploturl